Data fields
The data is separated between the differents signals recorded by the Dreem headband. Signal is shapes in windows of 30 seconds.

Electroencephalogram is measured at 7 differents locations of the head (eeg_1 -> eeg_7). The sampling frequency is 50 Hz.

Pulse oximeter (pulse_oximeter_infrared) and acceleromter (accelerometer_[x/y/z]) channels are sampled at 10Hz

eeg_1 - EEG in frontal position sampled at 50 Hz -> 1500 values
eeg_2 - EEG in frontal position sampled at 50 Hz -> 1500 values
eeg_3 - EEG in frontal position sampled at 50 Hz -> 1500 values
eeg_4 - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
eeg_5 - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
eeg_6 - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
eeg_7 - EEG in frontal-occipital position sampled at 50 Hz -> 1500 values
accelerometer_x - Accelerometer along x axis sampled at 10 Hz -> 300 values
accelerometer_y - Accelerometer along y axis sampled at 10 Hz -> 300 values
accelerometer_z - Accelerometer along z axis sampled at 10 Hz -> 300 values
pulse_oximeter_infrared - Pulse oximeter infrared channel sampled at 10 Hz -> 300 values

Filtrage des signaux avec critère de Nyquist et décomposition des EEG en bandes de fréquence.

In [2]:
import h5py
from scipy import signal
from tqdm import tqdm

In [2]:
train = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train.h5", "r")
train_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train_pp.h5", "w")
test = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test.h5", "r")
test_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test_pp.h5", "w")

In [3]:
list(train.keys())

['accelerometer_x',
 'accelerometer_y',
 'accelerometer_z',
 'eeg_1',
 'eeg_2',
 'eeg_3',
 'eeg_4',
 'eeg_5',
 'eeg_6',
 'eeg_7',
 'pulse_oximeter_infrared']

On laisse les données d'accéléromètre et oxymetre telles quelles.

In [4]:
train['accelerometer_x'].shape

(38289, 300)

In [5]:
for key in ['accelerometer_x',
            'accelerometer_y',
            'accelerometer_z',
            'pulse_oximeter_infrared']:
    print(key)
    current_shape = train[key].shape
    train_pp.create_dataset(key, shape = current_shape)
    for i in tqdm(range(current_shape[0])):
        train_pp[key][i] = train[key][i]

accelerometer_x


100%|██████████| 38289/38289 [10:54<00:00, 58.50it/s] 


accelerometer_y


100%|██████████| 38289/38289 [09:35<00:00, 66.53it/s] 


accelerometer_z


100%|██████████| 38289/38289 [13:49<00:00, 46.15it/s] 


pulse_oximeter_infrared


100%|██████████| 38289/38289 [13:36<00:00, 46.89it/s] 


In [6]:
for key in ['accelerometer_x',
            'accelerometer_y',
            'accelerometer_z',
            'pulse_oximeter_infrared']:
    print(key)
    current_shape = test[key].shape
    test_pp.create_dataset(key, shape = current_shape)
    for i in tqdm(range(current_shape[0])):
        test_pp[key][i] = test[key][i]

accelerometer_x


100%|██████████| 37439/37439 [24:46<00:00, 25.19it/s]  


accelerometer_y


100%|██████████| 37439/37439 [24:44<00:00, 25.23it/s]  


accelerometer_z


100%|██████████| 37439/37439 [27:20<00:00, 22.82it/s]  


pulse_oximeter_infrared


100%|██████████| 37439/37439 [24:15<00:00, 25.72it/s]  


In [7]:
train.close()
train_pp.close()
test.close()
test_pp.close()

On décompose les signaux EEG en différentes bandes:
- delta: 0-4 Hz
- Theta: 4-8 Hz
- Alpha: 8-13 Hz
- Beta: 13-22 Hz
- Gamme: > 30 Hz.

In [2]:
train = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train.h5", "r")
train_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train_pp.h5", "r+")
test = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test.h5", "r")
test_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test_pp.h5", "r+")

Delta

In [3]:
N, Wn = signal.buttord(wp = 4,
                       ws = 5, 
                       gpass = 3, 
                       gstop = 40, 
                       analog = False,
                       fs = 50)
sos = signal.butter(N, Wn, 
                    'lowpass', 
                    analog = False,
                    output='sos',
                   fs = 50)

In [7]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = train[current_eeg].shape
    train_pp.create_dataset(current_eeg + "_delta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        train_pp[current_eeg + "_delta"][j] = signal.sosfilt(sos, train[current_eeg][j])

eeg_1


100%|██████████| 38289/38289 [20:35<00:00, 31.00it/s]


eeg_2


100%|██████████| 38289/38289 [30:03<00:00, 21.23it/s]   


eeg_3


...

100%|██████████| 38289/38289 [19:58<00:00, 31.95it/s]


eeg_6


100%|██████████| 38289/38289 [19:53<00:00, 32.09it/s]


eeg_7


100%|██████████| 38289/38289 [21:01<00:00, 30.35it/s] 


In [15]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = test[current_eeg].shape
    test_pp.create_dataset(current_eeg + "_delta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        test_pp[current_eeg + "_delta"][j] = signal.sosfilt(sos, test[current_eeg][j])

eeg_1


100%|██████████| 37439/37439 [28:00<00:00, 22.28it/s]  


eeg_2


100%|██████████| 37439/37439 [23:42<00:00, 26.31it/s]  


eeg_3


...

100%|██████████| 37439/37439 [25:33<00:00, 24.41it/s] 


eeg_6


100%|██████████| 37439/37439 [20:52<00:00, 29.90it/s]


eeg_7


100%|██████████| 37439/37439 [22:28<00:00, 27.77it/s] 


Theta 4 8

In [16]:
N, Wn = signal.buttord(wp = [4, 8],
                       ws = [3, 9], 
                       gpass = 3, 
                       gstop = 40, 
                       analog = False,
                       fs = 50)
sos = signal.butter(N, Wn, 
                    'bandpass', 
                    analog = False,
                    output='sos',
                   fs = 50)

In [17]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = train[current_eeg].shape
    train_pp.create_dataset(current_eeg + "_theta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        train_pp[current_eeg + "_theta"][j] = signal.sosfilt(sos, train[current_eeg][j])

eeg_1


100%|██████████| 38289/38289 [19:48<00:00, 32.21it/s]


eeg_2


100%|██████████| 38289/38289 [19:51<00:00, 32.14it/s]


eeg_3


...

100%|██████████| 38289/38289 [19:46<00:00, 32.26it/s]


eeg_6


100%|██████████| 38289/38289 [22:53<00:00, 27.88it/s]  


eeg_7


100%|██████████| 38289/38289 [28:18<00:00, 22.54it/s]  


In [18]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = test[current_eeg].shape
    test_pp.create_dataset(current_eeg + "_theta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        test_pp[current_eeg + "_theta"][j] = signal.sosfilt(sos, test[current_eeg][j])

eeg_1


100%|██████████| 37439/37439 [31:21<00:00, 19.90it/s]  


eeg_2


100%|██████████| 37439/37439 [41:05<00:00, 15.18it/s]  


eeg_3


...

100%|██████████| 37439/37439 [19:16<00:00, 32.36it/s]


eeg_6


100%|██████████| 37439/37439 [19:20<00:00, 32.26it/s]


eeg_7


100%|██████████| 37439/37439 [30:39<00:00, 20.35it/s]  


In [19]:
train.close()
train_pp.close()
test.close()
test_pp.close()

In [3]:
train = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train.h5", "r")
train_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/train_pp.h5", "r+")
test = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test.h5", "r")
test_pp = h5py.File("../DreemSleepStageClassificationChallengeData/dreem-sleep-stages/test_pp.h5", "r+")

alpha 8-13

In [4]:
N, Wn = signal.buttord(wp = [8, 13],
                       ws = [7, 14], 
                       gpass = 3, 
                       gstop = 40, 
                       analog = False,
                       fs = 50)
sos = signal.butter(N, Wn, 
                    'bandpass', 
                    analog = False,
                    output='sos',
                   fs = 50)

In [5]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = train[current_eeg].shape
    train_pp.create_dataset(current_eeg + "_alpha", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        train_pp[current_eeg + "_alpha"][j] = signal.sosfilt(sos, train[current_eeg][j])

eeg_1


100%|██████████| 38289/38289 [27:38<00:00, 23.08it/s] 


eeg_2


100%|██████████| 38289/38289 [28:41<00:00, 22.24it/s]  


eeg_3


...

100%|██████████| 38289/38289 [32:44<00:00, 19.49it/s]  


eeg_6


100%|██████████| 38289/38289 [35:12<00:00, 18.12it/s]  


eeg_7


100%|██████████| 38289/38289 [44:02<00:00, 14.49it/s]  


In [6]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = test[current_eeg].shape
    test_pp.create_dataset(current_eeg + "_alpha", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        test_pp[current_eeg + "_alpha"][j] = signal.sosfilt(sos, test[current_eeg][j])

eeg_1


100%|██████████| 37439/37439 [42:16<00:00, 14.76it/s]  


eeg_2


100%|██████████| 37439/37439 [30:31<00:00, 20.45it/s]  


eeg_3


...

100%|██████████| 37439/37439 [30:09<00:00, 20.69it/s]  


eeg_6


100%|██████████| 37439/37439 [31:26<00:00, 19.84it/s]  


eeg_7


100%|██████████| 37439/37439 [35:38<00:00, 17.51it/s]  


beta 13 22 faire un passe haut plutot ?

In [7]:
N, Wn = signal.buttord(wp = 13,
                       ws = 12, 
                       gpass = 3, 
                       gstop = 40, 
                       analog = False,
                       fs = 50)
sos = signal.butter(N, Wn, 
                    'highpass', 
                    analog = False,
                    output='sos',
                   fs = 50)

In [8]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = train[current_eeg].shape
    train_pp.create_dataset(current_eeg + "_beta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        train_pp[current_eeg + "_beta"][j] = signal.sosfilt(sos, train[current_eeg][j])

eeg_1


100%|██████████| 38289/38289 [34:05<00:00, 18.72it/s]  


eeg_2


100%|██████████| 38289/38289 [32:52<00:00, 19.41it/s]  


eeg_3


...

100%|██████████| 38289/38289 [10:27:16<00:00,  1.02it/s]      


eeg_6


100%|██████████| 38289/38289 [21:05<00:00, 30.27it/s] 


eeg_7


100%|██████████| 38289/38289 [20:02<00:00, 31.85it/s]


In [9]:
num_eeg = 7
for i in range(num_eeg):
    current_eeg = 'eeg_' + str(i+1)
    print(current_eeg)
    current_shape = test[current_eeg].shape
    test_pp.create_dataset(current_eeg + "_beta", shape = current_shape)
    for j in tqdm(range(current_shape[0])):
        test_pp[current_eeg + "_beta"][j] = signal.sosfilt(sos, test[current_eeg][j])

eeg_1


100%|██████████| 37439/37439 [19:08<00:00, 32.61it/s]


eeg_2


100%|██████████| 37439/37439 [18:55<00:00, 32.99it/s] 


eeg_3


...

100%|██████████| 37439/37439 [21:05<00:00, 29.57it/s]  


eeg_6


100%|██████████| 37439/37439 [31:57<00:00, 19.52it/s]  


eeg_7


100%|██████████| 37439/37439 [19:11<00:00, 32.51it/s]


In [10]:
train.close()
train_pp.close()
test.close()
test_pp.close()